In [186]:
from PIL import Image
from direct.preprocessor import Preprocessor, pose_inv, SceneData
import numpy as np
import open3d as o3d

demo_rgb = np.array(Image.open("../data/lego/demo_rgb.png"))
demo_depth = np.array(Image.open("../data/lego/demo_depth.png"))
demo_mask = np.array(Image.open("../data/lego/demo_mask.png"))

live_rgb = np.array(Image.open("../data/lego/live_rgb.png"))
live_depth = np.array(Image.open("../data/lego/live_depth.png"))
live_mask = np.array(Image.open("../data/lego/live_mask.png"))
intrinsics = np.load("../handeye/intrinsics.npy")



In [187]:
data = SceneData(
    image_0=demo_rgb,
    image_1=live_rgb,
    depth_0=demo_depth,
    depth_1=live_depth,
    seg_0=demo_mask,
    seg_1=live_mask,
    intrinsics_0=intrinsics,
    intrinsics_1=intrinsics,
    T_WC=np.eye(4) # cam frame
)

In [188]:
processor = Preprocessor()
data.update(processor(data))

pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(data["pc1"][:, :3])
o3d.visualization.draw_geometries([pcd0, pcd1])

# Calculate the centroid of each point cloud
pcd0_centre = np.mean(data["pc0"][:, :3], axis=0)  # Calculate mean across columns (axis=0)
pcd1_centre = np.mean(data["pc1"][:, :3], axis=0)  # Calculate mean across columns (axis=0)

print(pcd0_centre)
print(pcd1_centre)
# # Compute the difference between the centroids
# diff = pcd0_centre - pcd1_centre
# diff

[0.26962662 0.04750962 0.7440325 ]
[0.08233145 0.07242211 0.6951443 ]


In [190]:
import open3d as o3d
import numpy as np
import copy
import cv2

# Assuming data["pc0"] and data["pc1"] are your point cloud data
pcd0 = o3d.geometry.PointCloud()
pcd1 = o3d.geometry.PointCloud()

pcd0.points = o3d.utility.Vector3dVector(data["pc0"][:, :3])
pcd1.points = o3d.utility.Vector3dVector(data["pc1"][:, :3])

# Estimate normals for each point cloud
pcd0.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=40))
pcd1.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=40))

# Function to draw registration results
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, target_temp])

# Compute FPFH features
voxel_size = 0.05  # Set voxel size for downsampling (adjust based on your data)
source_down = pcd0.voxel_down_sample(voxel_size)
target_down = pcd1.voxel_down_sample(voxel_size)

source_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
target_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    source_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    target_down,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

# Global registration using RANSAC
distance_threshold = voxel_size * 1.5
result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, source_fpfh, target_fpfh, mutual_filter=False,
    max_correspondence_distance=distance_threshold,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(), 
    ransac_n=4,
    checkers=[
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), 
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    ],
    criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
)

# Use the result of global registration as the initial transformation for ICP
trans_init = result.transformation

# Apply ICP
threshold = 0.02  # Set a threshold for ICP, this depends on your data
reg_p2p = o3d.pipelines.registration.registration_icp(
    pcd0, pcd1, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())

# Get the transformation matrix
T_delta_cam = reg_p2p.transformation

# Draw the result
draw_registration_result(pcd0, pcd1, T_delta_cam)

print(T_delta_cam)


[[ 0.79853988  0.33557628 -0.49972255  0.22515415]
 [-0.3333886   0.93778593  0.09700306  0.04674598]
 [ 0.5011847   0.08914099  0.86073677 -0.0846056 ]
 [ 0.          0.          0.          1.        ]]


In [191]:
T_delta_world = data["T_WC"] @ T_delta_cam @ pose_inv(data["T_WC"])
T_delta_world

array([[ 0.79853988,  0.33557628, -0.49972255,  0.22515415],
       [-0.3333886 ,  0.93778593,  0.09700306,  0.04674598],
       [ 0.5011847 ,  0.08914099,  0.86073677, -0.0846056 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [162]:
from scipy.spatial.transform import Rotation as R

def translation_from_matrix(matrix):
    """Extracts the translation vector from a 4x4 homogeneous transformation matrix."""
    return matrix[:3, 3]

def euler_from_matrix(matrix):
    """Extracts the quaternion from a 4x4 homogeneous transformation matrix."""
    rotation_matrix = matrix[:3, :3].copy()
    rotation = R.from_matrix(rotation_matrix)
    return rotation.as_euler(seq="XYZ", degrees=True)

trans = translation_from_matrix(T_delta_base)
rotation = euler_from_matrix(T_delta_base)

print(trans, rotation)

[0.14343193 0.41713163 0.01285603] [ 2.21225953e-02  1.46855369e+00 -3.23223851e+01]


In [133]:
PointCloud = np.ndarray

def rotate_pointcloud(pcd: PointCloud, angle_z: float):
    print("predicted rotation", np.rad2deg(angle_z))
    R = np.eye(3)
    cosine = np.cos(angle_z)
    sine = np.sin(angle_z)
    R[0, 0] = cosine
    R[1, 1] = cosine
    R[0, 1] = -sine
    R[1, 0] = sine

    pcd[:3, :] = R @ pcd[:3, :]
    return R, pcd

def find_translation(pcd0: PointCloud, pcd1: PointCloud) -> np.ndarray:
    pcd0_centre = np.mean(pcd0[:3, :], axis=1)
    pcd1_centre = np.mean(pcd1[:3, :], axis=1)
    return pcd1_centre - pcd0_centre
    
R_mtx, rotated_pcd0 = rotate_pointcloud(data["pc0"], rotation[-1])
translation = find_translation(rotated_pcd0, data["pc1"])

print(translation)
T_delta_base = np.eye(4)
T_delta_base[:3, :3] = R_mtx
T_delta_base[:3, 3] = translation

T_delta_cam = pose_inv(data["T_WC"]) @ T_delta_base @ data["T_WC"]

print(T_delta_base)

predicted rotation -1854.694161068739
[-0.09844735 -0.39142355 -0.21084177]
[[ 0.57794079  0.8160787   0.         -0.09844735]
 [-0.8160787   0.57794079  0.         -0.39142355]
 [ 0.          0.          1.         -0.21084177]
 [ 0.          0.          0.          1.        ]]
